# Problema 8


Se desean medir los colores (U, B, V) de una estrella desconocida. Se dispone de un
fotómetro cuyas "cuentas" por minuto son proporcionales al flujo recibido. Se mide en la estrella
standard Vega durante 1 minuto a diferentes distancias cenitales z, obteniéndose las siguientes
cuentas para los diferentes filtros:
Z
V
B
U
20º
104
8x103
5x103
40º
9460
7247
4223
60º
8063
5451
2597
Posteriormente se miden los flujos para la estrella desconocida, que es mucho más débil que
Vega, midiéndose en cada filtro durante 10 minutos. Se obtienen los siguientes registros:
Z
V
B
U
40º
3784
2998
1689
Determinar los valores de mV, mB y mU, así como apreciar cualitativamente el color de la
estrella; ¿qué tipo de estrella será si se encuentra en la Secuencia Principal?.
